In [29]:
import pandas as pd
import numpy as np
import json
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import rdChemReactions
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.Draw import IPythonConsole
from IPython.display import SVG, display
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
from rdkit.Chem import MACCSkeys
from rdkit.Chem import AllChem
import pandas as pd
import rdchiral
import itertools
import pickle
import sys
from tqdm import tqdm 
from rdchiral.main import rdchiralRun, rdchiralRunText, rdchiralReaction, rdchiralReactants
import cobra
from itertools import chain, combinations
import sys
import multiprocessing
from functools import partial
import matplotlib.pyplot as plt
import multiprocessing
from tqdm import tqdm
sys.path.append('../retrosynthesis/')

from common import *



### input and output

In [30]:
# step1
#input
mnxreac_smile_atom_mapping_rules_file_path = '../../Data/rules/MNXreaction_smiles_atommap_rules_filter.csv'
yeast870_path = '../../Data/model/yeast-GEM-final.csv'
new_met_info_to_GEM_path = '../../Results/not_lipid/top50_0.3_re/metabolites_info_to_GEM_top50_0.3.csv'
target_100met_path = '../../Data/100met_with_smiles.xlsx'


#output
# rxndb_path = '../../Results/heterologous/heterologous_RXNdb.json'
# newdbSmiles_df_path = '../Data/retrosynthesis/heterologous_NewdbSmiles.pickle'
# failedrxn_path = '../../Results/heterologous/heterologous_failedrxn.pickle'
# retrosys_smiles_calculate_similarity_file_path = '../../Results/heterologous/heterologous_retrosys_smiles_calculate_similarity.json'
heterologous_met_byproduct_result_path = '../../Data/analysis/heterologous_met_byproduct_result.csv'
source_fig5b_path = '../../Results/source_data/fig5-b.csv'
yeast8U_heterologous_product_reaction_figure_path = '../../Results/Figures/fig5-b.pdf'

In [3]:
def process_rule(rule_smiles, mets_total_smiles):
    score = []
    smiles = []
    if 'C' in rule_smiles or 'c' in rule_smiles:
        for j in mets_total_smiles:
            current_score = calculate_similarity(rule_smiles, j)
            if current_score > 0.3:
                score.append(current_score)
                smiles.append(j)
    else:
        smiles.append(rule_smiles)
        score.append(1)

    if len(score) == 0:
        return rule_smiles, [], []
    elif len(score) > 40:
        top40_data = sorted(zip(score, smiles), reverse=True)[:40]
    else:
        top40_data = sorted(zip(score, smiles), reverse=True)
    
    top40_scores, top40_smiles = zip(*top40_data)
    
    return rule_smiles, list(top40_smiles), list(top40_scores)

In [4]:
def process_retrorule(index_row, mets_total_smiles,retrosys_smiles_calculate_similarity_pd,heterologous_met_smiles):
    rxndb_list_tmp = []
    # newdbSmiles_tmp = []
    failedrxn_tmp = []
    index, row = index_row
    rxn_id, ECnumber, reactants, products = row['MNX_ID'], row['classifs'], row['substrate_smiles'], row['product_smiles']
    deprecated_equ_smiles, reactantSMARTs, productSMARTs, rule = row['deprecated_equ_smiles'], row['ReactantsSMARTs'], row['ProductSMARTs'], row['RetroRules']
    deprecated_equ_smiles = deprecated_equ_smiles.split('>>')
    if rxn_id not in ['MNXR101882', 'MNXR101884', 'MNXR101879', 'MNXR101885', 'MNXR101886', 'MNXR101887',
                        'MNXR103528', 'MNXR103575']:  # ATP NADH conversion stuff
        productSMARTs = productSMARTs.split('.')
        products = products.split('.')
        new_products_smiles = []
        scores = []

        if len(productSMARTs) == len(products):
            for x in range(len(productSMARTs)):  # prepare mets with substructure and sort the reactants combination
                new_products_smile_one,score_one = get_calculate_similarity(products[x],retrosys_smiles_calculate_similarity_pd)
                new_products_smiles.append(new_products_smile_one)
                scores.append(score_one)

            product_prepared = []
            scores_prepared = []
            for lst, score_lst in zip(new_products_smiles, scores):
                if heterologous_met_smiles in lst:
                    if len(new_products_smiles) == 1:  # if only one sub list
                        product_prepared.append([heterologous_met_smiles])
                        scores_prepared.append(score_lst)
                    else:
                        other_lists = [sub_lst for sub_lst in new_products_smiles if sub_lst != lst]
                        tmp_product = list(itertools.product([heterologous_met_smiles], *other_lists))
                        product_prepared += tmp_product

                        tmp_scores = list(itertools.product([score_lst[new_products_smiles.index(lst)]], *scores))
                        scores_prepared += tmp_scores

            for m in range(0, len(product_prepared)):  # apply the rule and output the rxn
                smile = product_prepared[m]
                score = scores_prepared[m]
                
                if len(smile) > 1:
                    product_smile = '.'.join(smile)
                else:
                    product_smile = smile[0]

                retrorule_tmp = rule.split('>>')
                retrorule_tmp = '(' + retrorule_tmp[0] + ')>>' + '(' + retrorule_tmp[1] + ')'            
                product_smile_tmp = product_smile
                try:
                    # print('product_smile_tmp',product_smile_tmp)
                    product_mol = Chem.MolFromSmiles(product_smile_tmp)
                    reaction = AllChem.ReactionFromSmarts(retrorule_tmp)
                    reactant_smiles = reaction.RunReactants((product_mol,))
                    smiles = [Chem.MolToSmiles(x[0]) for x in reactant_smiles]
                    reactant_smiles = list(set(smiles))

                    for reactant_smile in reactant_smiles:
                        if len(deprecated_equ_smiles) > 0:
                            if deprecated_equ_smiles[0] == '':
                                reactant_smile_final = reactant_smile
                            else:
                                reactant_smile_final = reactant_smile + '.' + deprecated_equ_smiles[0]
                            if deprecated_equ_smiles[1] == '':
                                product_smile_final = product_smile_tmp
                            else:
                                product_smile_final = product_smile_tmp + '.' + deprecated_equ_smiles[1]
                        rxn_smiles = reactant_smile + '>>' + product_smile_tmp
                        rxn_smiles_final = reactant_smile_final + '>>' + product_smile_final
                        
                        tmp = {
                            'EC number': ECnumber,
                            'rule': rule,
                            'templateID': rxn_id,
                            'templateSubstrate': products,
                            'rxn_smiles_basic': rxn_smiles,
                            'rxn_smiles_final': rxn_smiles_final,
                            'reactant_smile': reactant_smile,
                            'productsmile': product_smile_tmp,
                            'similarity': score
                        }
                        if check_rule(tmp['rxn_smiles_basic'])=='blance':
                            rxndb_list_tmp.append(tmp)  
                            # mets_tmp = reactant_smile.split('.')      
                            # for reactant in mets_tmp:             
                            #     if reactant not in mets_total_smiles and reactant not in newdbSmiles_tmp:         
                            #         newdbSmiles_tmp.append(reactant)        
                except:
                    failedrxn_tmp.append(rxn_id)
                
    return(rxndb_list_tmp,failedrxn_tmp)

In [11]:
rules_product_smiles_lst = []
retrorules = pd.read_csv(mnxreac_smile_atom_mapping_rules_file_path)
for index,row in retrorules.iterrows():
    tmp  = row['product_smiles'].split('.')
    for i in tmp:
        if i not in rules_product_smiles_lst:
            rules_product_smiles_lst.append(i)
print('rules_product_smiles_lst',len(rules_product_smiles_lst))

rules_product_smiles_lst 9251


In [7]:
target_100met = pd.read_excel(target_100met_path)
target_100met = target_100met[target_100met['Group']=='heterologous']
target_100met = target_100met[~target_100met['Smiles'].isna()]
target_100met.reset_index(drop=True,inplace=True)
target_100met['byproduct'] = ' '
target_100met['EC number'] = ' '
target_100met.head()

,Name,Formula,KEGG ID,CHEBI ID,Group,class,Gene target,Smiles,byproduct,EC number
0,n-butanol,C4H10O,C06142,28885,heterologous,alcohol,Link,CCCCO,,
1,(S)-Reticuline,C19H23NO4,C02105,16718,heterologous,alkaloid,Link,COc1ccc(C[C@H]2c3cc(O)c(OC)cc3CCN2C)cc1O,,
2,psilocybin,C12H17N2O4P,C07576,8614,heterologous,alkaloid,Link,CN(C)CCc1c[nH]c2cccc(OP(=O)(O)O)c12,,
3,Nicotianamine,C12H21N3O6,C05324,17721,heterologous,amino acid,Link,N[C@@H](CCN[C@@H](CCN1CC[C@H]1C(=O)O)C(=O)O)C(...,,
4,4-hydroxymandelic,C8H8O4,C11527,16388,heterologous,aromatic,Link,O=C(O)C(O)c1ccc(O)cc1,,


In [3]:
# target_100met = pd.read_csv('../Data/analysis/heterologous_met_byproduct_result.csv')
# len(target_100met)
# target_100met

,Name,Formula,KEGG ID,CHEBI ID,Group,class,Gene target,Smiles,byproduct,EC number
0,L-phenylacetylcarbinol,C9H10O2,NaN,NaN,heterologous,alcohol,Link,CC(=O)[C@H](C1=CC=CC=C1)O,7,"['1.14.13.7', '1.14.13.7', '1.14.13.7', '1.14...."
1,n-butanol,C4H10O,C06142,28885.0,heterologous,alcohol,Link,CCCCO,5,"['1.1.3.17', '1.14.13.230', '1.14.19.48', '1.1..."
2,(S)-Reticuline,C19H23NO4,C02105,16718.0,heterologous,alkaloid,Link,CN1CCC2=CC(=C(C=C2[C@@H]1CC3=CC(=C(C=C3)OC)O)O)OC,7,"['1.14.14.1', '1.14.14.1', '1.14.14.1', '1.14...."
3,psilocybin,C12H17N2O4P,C07576,8614.0,heterologous,alkaloid,Link,CN(C)CCC1=CNC2=C1C(=CC=C2)OP(=O)(O)O,7,"['1.14.13.8', '1.14.14.-;1.14.14.1', '1.14.14...."
4,Nicotianamine,C12H21N3O6,C05324,17721.0,heterologous,amino acid,Link,C1CN([C@@H]1C(=O)O)CC[C@@H](C(=O)O)NCC[C@@H](C...,6,"['3.1.3.15', '3.1.3.15', '3.1.3.15', '1.14.13...."
5,4-hydroxymandelic,C8H8O4,C11527,16388.0,heterologous,aromatic,Link,C1=CC(=CC=C1C(C(=O)O)O)O,11,"['1.14.13.29;1.14.13.n7', '1.14.13.29;1.14.13...."
6,cinnamate,C9H8O2,C00423,15669.0,heterologous,aromatic,Link,C1=CC=C(C=C1)/C=C/C(=O)[O-],8,"['1.2.1.3', '1.14.13.33', '1.14.13.33', '1.14...."
7,mandelic,C8H8O3,C01984,17756.0,heterologous,aromatic,Link,C1=CC=C(C=C1)[C@@H](C(=O)O)O,7,"['1.14.13.7', '1.14.13.7', '1.14.13.7', '1.14...."
8,p-coumaric acid,C9H8O3,C00811,32374.0,heterologous,aromatic,Link,C1=CC(=CC=C1/C=C/C(=O)O)O,10,"['1.14.13.29;1.14.13.n7', '1.14.13.29;1.14.13...."
9,rosmarinate,C18H16O8,C01850,50371.0,heterologous,aromatic,Link,C1=CC(=C(C=C1C[C@H](C(=O)O)OC(=O)/C=C/C2=CC(=C...,19,"['1.13.11.-;1.13.11.29', '1.13.11.-;1.13.11.29..."


## byproduct prediction

In [13]:
def check_productsmile(smiles):#only keep smiles with carbon
    smiles_lst = smiles.split('.')
    smiles_lst = [x for x in smiles_lst if smiles_has_carbon(x)]
    return len(smiles_lst)

In [21]:
for index,row in tqdm(target_100met.iterrows(),total=len(target_100met)):
    # if pd.isna(row['byproduct']):
    if index == 0:
        continue
    if row['byproduct']==' ':
        print(row['Name'])
        heterologous_met_smiles = row['Smiles']
        heterologous_met_smiles = normalize_smiles(heterologous_met_smiles)

        model_met = get_GEM_all_smiles(yeast870_path)
        new_met_info_to_GEM = pd.read_csv(new_met_info_to_GEM_path,index_col=None)
        new_met_info_to_GEM.head(6)
        new_met = new_met_info_to_GEM[new_met_info_to_GEM['ID'].apply(lambda x:'sn_' in x)]['new_met_smiles'].to_list()
        new_met = [normalize_smiles(x) for x in new_met if 'C' in x or 'c' in x]
        # add heterologous_met_smiles to model_met list
        new_model_and_heterologous_met = model_met + new_met
        new_model_and_heterologous_met.append(heterologous_met_smiles)
        new_model_and_heterologous_met = [smile for smile in new_model_and_heterologous_met if '.' not in smile]
        new_model_and_heterologous_met = [smile for smile in new_model_and_heterologous_met if 'C' in smile or 'c' in smile] ###只筛第一轮
        print(len(new_model_and_heterologous_met))
        #calculate similarity
        num_processes = 30
        pool = multiprocessing.Pool(num_processes)
        retrosys_smiles_calculate_similarity = {'rules_smiles': [],
                                                'smiles_in_mets_total_smiles': [],
                                                'scores': []}


        process_rule_partial = partial(process_rule, mets_total_smiles=new_model_and_heterologous_met)
        for result in tqdm(pool.imap(process_rule_partial, rules_product_smiles_lst, chunksize=50), total=len(rules_product_smiles_lst)):
            if result[0] not in retrosys_smiles_calculate_similarity['rules_smiles']:
                retrosys_smiles_calculate_similarity['rules_smiles'].append(result[0]) 
                retrosys_smiles_calculate_similarity['smiles_in_mets_total_smiles'].append(result[1]) 
                retrosys_smiles_calculate_similarity['scores'].append(result[2]) 
        pool.close()
        pool.join()

        retrosys_smiles_calculate_similarity_pd = pd.DataFrame(retrosys_smiles_calculate_similarity)

        rxndb_list = []
        # newdbSmiles = []
        failedrxn = []
        num_processes =30  #  adjust the number of processes according to your needs
        pool = multiprocessing.Pool(num_processes)
        # Execute in parallel using process pool

        # Create a partial function that passes mets_total_smiles and mets_total_mol as default parameters to the process_retrorule function
        process_retrorule_partial = partial(process_retrorule, mets_total_smiles=new_model_and_heterologous_met,retrosys_smiles_calculate_similarity_pd = retrosys_smiles_calculate_similarity_pd,heterologous_met_smiles = heterologous_met_smiles)

        # Using the process pool to execute in parallel, the chunksize parameter is placed inside the imap function.
        for result in tqdm(pool.imap(process_retrorule_partial, retrorules.iterrows(), chunksize=50), total=len(retrorules)):
            # if result == ([], [], []):
            #     pass
            # else:
            #     print(result)
            if result[0] :  #  check if the result is not empty
                rxndb_list.extend(result[0]) 
            # if result[1] is not None:
            #     newdbSmiles.extend(result[1])
            if result[1]:
                failedrxn.extend(result[1])  

        pool.close()
        pool.join()

        rxndb = {}
        for i, rxn_dict in enumerate(rxndb_list):
            # generate a unique key for each reaction, for example "rxn0"，"rxn1"，"rxn2"，...
            key = f'rxn{i+1}'
            # merge to the dictionary
            rxndb[key] = rxn_dict
        # with open(rxndb_path, 'w') as json_file:
        #     json.dump(rxndb, json_file)

        # with open(rxndb_path, 'r') as f:
        #     rxndb = json.load(f)
        print('rxndb num',len(rxndb))
        # print('newdbSmiles num',len(newdbSmiles))
        print('failedrxn num',len(failedrxn))

        miss_smiles = ['C1C=CN(C=C1C(=O)N)C2C(C(C(O2)COP(=O)(O)OP(=O)(O)OCC3C(C(C(O3)N4C=NC5=C(N=CN=C54)N)O)O)O)O',#NADH
                        'C1C=CN(C=C1C(=O)N)C2C(C(C(O2)COP(=O)(O)OP(=O)(O)OCC3C(C(C(O3)N4C=NC5=C(N=CN=C54)N)OP(=O)(O)O)O)O)O']#NADPH
        miss_reaction_lst = []
        for k,v in rxndb.items():
            reactant_smile_lst = v['rxn_smiles_final'].split('>>')[0].split('.')
            if any(compare_smiles_inchikey(miss_smiles[0],met) for met in reactant_smile_lst):
                # print(k,v['rxn_smiles_final'])
                miss_reaction_lst.append(k)
            if any(compare_smiles_inchikey(miss_smiles[1],met) for met in reactant_smile_lst):
                # print(k,v['rxn_smiles_final'])    
                miss_reaction_lst.append(k)
        print(len(miss_reaction_lst))
        
        result_df = {'ID':[],
                    'reactant_smile':[],
                    'productsmile':[]}
        for k,v in rxndb.items():
            result_df['ID'].append(k)
            result_df['reactant_smile'].append(v['reactant_smile'])
            result_df['productsmile'].append(v['productsmile'])
        result_df = pd.DataFrame({'ID': result_df['ID'],
                                'reactant_smile': result_df['reactant_smile'],
                                'productsmile': result_df['productsmile']})
        result_df.drop_duplicates(subset=['reactant_smile', 'productsmile'], inplace=True)
        result_df.reset_index(drop=True, inplace=True)

        result_df = result_df[result_df['productsmile']!=heterologous_met_smiles+'.OO']
        result_df = result_df[result_df['productsmile']!=heterologous_met_smiles+'.O=C=O']
        result_df = result_df[result_df['productsmile']!=heterologous_met_smiles+'.O=S([O-])[O-]']
        result_df = result_df[result_df['productsmile']!=heterologous_met_smiles+'.[SH-]']
        result_df = result_df[result_df['productsmile']!=heterologous_met_smiles+'.O=P([O-])([O-])OP(=O)([O-])O']
        result_df = result_df[result_df['productsmile']!=heterologous_met_smiles+'.II']
        result_df = result_df[result_df['productsmile']!=heterologous_met_smiles+'.[NH4+]']

        unique_smiles = []
        for i in result_df['reactant_smile']:
            if all(compare_smiles_inchikey(x,i)!=1 for x in unique_smiles):
                unique_smiles.append(i)
            else:
                pass
        result_df = result_df[result_df['reactant_smile'].isin(unique_smiles)]

        unique_productsmile = []
        for i in result_df['productsmile']:
            if i.replace('O=P(O)(O)','') not in unique_productsmile:
                unique_productsmile.append(i)
            else:
                pass
        result_df = result_df[result_df['productsmile'].isin(unique_productsmile)]

        result_df['productsmile_num'] = result_df['productsmile'].apply(lambda x: check_productsmile(x))
        result_df = result_df[result_df['productsmile_num']==1]
        result_df = result_df[~result_df['ID'].isin(miss_reaction_lst)]

        res = result_df['reactant_smile'].to_list()
        ec_lst = []
        for k,v in rxndb.items():
            if v['reactant_smile'] in res:
                ec_lst.append(v['EC number'])

        print(row['Name'],len(result_df))
        print(row['Name'],ec_lst)

        target_100met.loc[index,'byproduct'] = len(result_df)
        # target_100met.loc[index,'EC number'] = ec_lst
        target_100met.loc[index,'EC number'] = str(ec_lst)
        target_100met.loc[index,'byproduct'] = str(res)

        
        target_100met.to_csv(heterologous_met_byproduct_result_path,index=None)

  0%|          | 0/40 [00:00<?, ?it/s]

(S)-Reticuline
2141


100%|██████████| 20210/20210 [00:28<00:00, 708.61it/s]


rxndb num 1273
failedrxn num 0
16


  5%|▌         | 2/40 [19:58<6:19:23, 599.05s/it]

...

_-Amyrin 28
_-Amyrin ['1.14.14.1;1.14.14.25;1.14.14.55;1.14.14.57', '1.14.13.-', '1.1.3.16', '1.14.14.1', '1.14.14.1', '1.14.14.1', '1.14.14.1', '1.14.14.1', '1.14.14.1', '1.14.14.1', '1.14.14.1', '1.14.14.1', '1.14.14.1', '1.14.13.-', '1.14.13.72', '1.14.13.72', '1.14.13.-', '1.14.13.-', '1.14.13.-', '1.14.13.-', '1.14.13.-', '1.14.13.-', '1.14.13.-', '1.14.13.-', '1.14.13.-', '1.14.13.-', '5.3.3.5', '1.14.19.21', '1.14.19.21', '5.3.3.5', '1.3.1', '1.14.13;1.14.13.-', '1.14.13;1.14.13.-', '1.14.13;1.14.13.-', '1.14.13;1.14.13.-', '1.14.14.133', '5.3.3', '5.3.3', '1.14.13', '1.14.13', '1.14.13', '1.14.13', '1.14.13', '1.14.13', '1.14.13', '1.14.13', '1.14.13', '1.14.13', '1.14.13', '1.14.13', '5.3.3.-;5.3.3.5', '1.14.13;1.14.13.-', '1.14.13;1.14.13.-', '1.14.13;1.14.13.-', '1.14.13;1.14.13.-', '1.14.13;1.14.13.-', '1.14.13;1.14.13.-', '1.14.13;1.14.13.-', '1.14.13;1.14.13.-', '1.14.13;1.14.13.-', '1.14.13;1.14.13.-', '1.14.13;1.14.13.48', '1.14.14.1', '1.14.13.126;1.14.15.16', '1.14.13

100%|██████████| 20210/20210 [00:33<00:00, 604.19it/s]


rxndb num 145
failedrxn num 0


100%|██████████| 40/40 [12:52:22<00:00, 1158.57s/it]

3
_-ionone 10
_-ionone ['1.1.1;1.1.1.207', '1.1.1;1.1.1.208', '1.1.1.275', '1.1.1.296', '1.1.1.296', '1.1.1.296', '1.1.1.296', '1.1.1.296', '1.1.1.296', '1.1.1.296', '1.14.14.-;1.14.14.1', '1.1.1.323', '1.1.1.323', '1.14.13.229', '1.14.13.229', '1.14.13.229', '1.14.13', '1.1.1;1.1.1.-', '1.14.13;1.14.13.48', '1.1.1.1', '1.1.1.1', '1.1.1.1', '1.6.99.1', '1.3.1', '1.14.13.49', '1.14.13.49', '1.14.13;1.14.14.1', '1.1.1.198', '1.1.1.227', '1.14.13', '1.14.13', '1.14.13.-;1.14.13.119;1.14.14.149', '1.1.1.275', '1.14.13.-;1.14.14.149', '1.1.1.322', '1.1.1.322', '1.1.1.323', '1.1.1.323', '1.1.1.M48', '1.1.1', '1.1.1', '1.1.1.227', '1.14.13.133', '1.14.13;1.14.13.143', '1.1.1.323', '1.1.1.323', '1.14.13.-', '1.14.13.-', '1.14.99.45', '1.14.13.48', '1.14.13.119', '1.1.1.-', '1.1.1.-', '1.1.1.-;1.1.1.1;1.1.1.327', '1.1.1.296', '1.14.13.106', '1.14.13.80;1.14.14.53']


In [22]:
target_100met

,Name,Formula,KEGG ID,CHEBI ID,Group,class,Gene target,Smiles,byproduct,EC number
0,n-butanol,C4H10O,C06142,28885,heterologous,alcohol,Link,CCCCO,"['C=CCCO.O.O', 'O.OCCCCO', 'CCCC=O.OO']","['1.14.19.48', '1.14.13.230', '1.1.3.20', '1.1..."
1,(S)-Reticuline,C19H23NO4,C02105,16718,heterologous,alkaloid,Link,COc1ccc(C[C@H]2c3cc(O)c(OC)cc3CCN2C)cc1O,['COc1cc2c(cc1O)C(Cc1ccc(OC)c(O)c1O)N(C)CC2.O'...,"['1.14.14.-;1.14.14.1', '1.14.14.-;1.14.14.1',..."
2,psilocybin,C12H17N2O4P,C07576,8614,heterologous,alkaloid,Link,CN(C)CCc1c[nH]c2cccc(OP(=O)(O)O)c12,['CN(C)CCc1c[nH]c2cccc([O-])c12.O=P([O-])([O-]...,['3.1;3.1.3.5;3.1.4.1;3.1.4.16;3.1.4.17;3.1.4....
3,Nicotianamine,C12H21N3O6,C05324,17721,heterologous,amino acid,Link,N[C@@H](CCN[C@@H](CCN1CC[C@H]1C(=O)O)C(=O)O)C(...,[],[]
4,4-hydroxymandelic,C8H8O4,C11527,16388,heterologous,aromatic,Link,O=C(O)C(O)c1ccc(O)cc1,"['O=C(O)C(O)c1cc(O)c(O)cc1O', 'O.O=C(O)C(O)c1c...","['1.10.3.1;1.14.18.1', '1.10.3.1;1.14.18.1', '..."
5,cinnamate,C9H8O2,C00423,15669,heterologous,aromatic,Link,O=C(O)/C=C/c1ccccc1,"['O.O=C(O)C=Cc1ccc(O)cc1', 'O.O=C(O)C=Cc1ccccc...","['1.14.13.189', '1.14.13.189', '1.14.13.189', ..."
6,mandelic,C8H8O3,C01984,17756,heterologous,aromatic,Link,O=C(O)[C@@H](O)c1ccccc1,"['O.O=C(O)C(O)c1cccc(O)c1', 'O.O=C(O)C(O)c1ccc...","['1.14.14.91', '1.14.14.91', '1.14.14.91', '1...."
7,p-coumaric acid,C9H8O3,C00811,32374,heterologous,aromatic,Link,O=C(O)/C=C/c1ccc(O)cc1,"['O.O=C(O)C=Cc1ccc(O)c(O)c1', 'O.O=C(O)C=Cc1cc...","['1.14.13.29;1.14.13.n7', '1.14.13.29;1.14.13...."
8,rosmarinate,C18H16O8,C01850,50371,heterologous,aromatic,Link,O=C(/C=C/c1ccc(O)c(O)c1)O[C@H](Cc1ccc(O)c(O)c1...,['O=C(C=Cc1ccc(O)cc1)OC(Cc1cccc(O)c1)C(=O)O.O=...,"['1.10.3.1;1.14.18.1', '1.10.3.1;1.14.18.1', '..."
9,Salidroside,C14H20O7,C06046,9009,heterologous,aromatic,Link,OC[C@H]1O[C@@H](OCCc2ccc(O)cc2)[C@H](O)[C@@H](...,['O.O=P(O)(O)OC1C(CO)OC(OCCc2ccc(O)cc2)C(O)C1O...,"['3.1.3.69', '3.1.3.69', '3.1.3.69', '3.1.3.69..."


In [23]:
def get_ec_set(ec_lst):
    ec_set = []
    for x in ec_lst:
        ec_set +=x.split(';')
    ec_set = ['.'.join(x.split('.')[0:3]) for x in ec_set]
    ec_set = list(set(ec_set))
    ec_set = [x for x in ec_set if '-' not in x]
    return ec_set

In [24]:


target_100met_df = target_100met[['Name','class','byproduct','EC number']]
target_100met_df['byproduct_num'] = target_100met_df['byproduct']
# target_100met_df['EC number'] = target_100met_df['EC number'].apply(lambda x:literal_eval(x))
# target_100met_df['EC number'] = target_100met_df['EC number'].apply(lambda x:get_ec_set(x))
# target_100met_df['EC number_num'] = target_100met_df['EC number'].apply(lambda x:len(x))
# target_100met_df = target_100met_df[['Name','class','byproduct_num','EC number','EC number_num']]
classes_order = ['terpene', 'aromatic', 'flavonoid','organic acid','alcohol','fatty acids and lipids','other','alkaloid','amino acid']
classes_order = ['amino acid', 'alkaloid', 'other','fatty acids and lipids','alcohol','organic acid','flavonoid','aromatic','terpene']
target_100met_df = target_100met_df.sort_values(by='class', key=lambda x: pd.Categorical(x, categories=classes_order, ordered=True), na_position='last')
target_100met_df = target_100met_df.groupby('class', sort=False).apply(lambda x: x.sort_values(by='byproduct_num', ascending=True)).reset_index(drop=True)
target_100met_df.drop_duplicates(subset=['Name'], keep='first', inplace=True)

target_100met_df

/tmp/ipykernel_2600174/2876889681.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_100met_df['byproduct_num'] = target_100met_df['byproduct']


,Name,class,byproduct,EC number,byproduct_num
0,Nicotianamine,amino acid,[],[],[]
1,psilocybin,alkaloid,['CN(C)CCc1c[nH]c2cccc([O-])c12.O=P([O-])([O-]...,['3.1;3.1.3.5;3.1.4.1;3.1.4.16;3.1.4.17;3.1.4....,['CN(C)CCc1c[nH]c2cccc([O-])c12.O=P([O-])([O-]...
2,(S)-Reticuline,alkaloid,['COc1cc2c(cc1O)C(Cc1ccc(OC)c(O)c1O)N(C)CC2.O'...,"['1.14.14.-;1.14.14.1', '1.14.14.-;1.14.14.1',...",['COc1cc2c(cc1O)C(Cc1ccc(OC)c(O)c1O)N(C)CC2.O'...
3,Ethylene,other,['C1CO1.O'],['1.14.13.69'],['C1CO1.O']
4,"1,3,5-trihydroxyxanthone",other,"['O.O=c1c2c(O)cc(O)cc2oc2c(O)ccc(O)c12', 'O.O=...","['1.14.14.1', '1.14.14.1', '1.1.1.252', '1.14....","['O.O=c1c2c(O)cc(O)cc2oc2c(O)ccc(O)c12', 'O.O=..."
5,resveratrol,other,"['O.Oc1cc(O)cc(C=Cc2ccc(O)c(O)c2)c1', 'O.Oc1cc...","['1.14.13.7', '1.14.13.7', '1.14.13.7', '1.14....","['O.Oc1cc(O)cc(C=Cc2ccc(O)c(O)c2)c1', 'O.Oc1cc..."
6,DHA,fatty acids and lipids,"['CCC=CCC=CC/C=C\\C=C\\C=C\\CCCC=CCCC(=O)O', '...","['5.3.3.13', '5.3.3.13', '5.3.3.13', '5.3.3.13...","['CCC=CCC=CC/C=C\\C=C\\C=C\\CCCC=CCCC(=O)O', '..."
7,EPA,fatty acids and lipids,"['CCC=CCC=CCCC/C=C/C=C/C=C\\CCCC(=O)O', 'CCC=C...","['5.3.3.13', '5.3.3.13', '5.3.3.13', '5.3.3.13...","['CCC=CCC=CCCC/C=C/C=C/C=C\\CCCC(=O)O', 'CCC=C..."
8,n-butanol,alcohol,"['C=CCCO.O.O', 'O.OCCCCO', 'CCCC=O.OO']","['1.14.19.48', '1.14.13.230', '1.1.3.20', '1.1...","['C=CCCO.O.O', 'O.OCCCCO', 'CCCC=O.OO']"
9,Itaconic acid,organic acid,"['C=C(C=O)CC(=O)O.O', 'C=C(CC=O)C(=O)O.O', 'O=...","['1.2.1;1.2.1.-', '1.2.1;1.2.1.-', '4.2.1.11',...","['C=C(C=O)CC(=O)O.O', 'C=C(CC=O)C(=O)O.O', 'O=..."


In [25]:
target_100met_df['class'].value_counts()

class
terpene                   15
aromatic                   7
flavonoid                  5
other                      3
organic acid               3
alkaloid                   2
fatty acids and lipids     2
amino acid                 1
alcohol                    1
Name: count, dtype: int64

# figure

In [26]:
target_100met_df.to_csv(source_fig5b_path,index=None)

In [31]:
target_100met_df = pd.read_csv(source_fig5b_path)

In [ ]:


# Setting General Parameters
plt.figure(figsize=(1.5, 4.5), dpi=400)
plt.rcParams.update({'font.size': 6})
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['pdf.fonttype'] = 42

# Setting Axis Styles
plt.gca().spines['top'].set_linewidth(0)
plt.gca().spines['bottom'].set_linewidth(0.5)
plt.gca().spines['left'].set_linewidth(0.5)
plt.gca().spines['right'].set_linewidth(0)

# Plot horizontal bar charts, swapping x and y axes
bar_width = 0.6
index = np.arange(len(target_100met_df['Name']))
plt.barh(index[28:44], target_100met_df['byproduct_num'][28:44], height=bar_width, label='Terpene', color='#a6cee3')
plt.barh(index[21:28], target_100met_df['byproduct_num'][21:28], height=bar_width, label='Aromatic', color='#1f78b4')
plt.barh(index[16:21], target_100met_df['byproduct_num'][16:21], height=bar_width, label='Flavonoid', color='#b2df8a')
plt.barh(index[12:16], target_100met_df['byproduct_num'][12:16], height=bar_width, label='Organic acid', color='#33a02c')
plt.barh(index[9:12], target_100met_df['byproduct_num'][9:12], height=bar_width, label='Other', color='#fb9a99')
plt.barh(index[6:9], target_100met_df['byproduct_num'][6:9], height=bar_width, label='Fatty acids', color='#e31a1c')
plt.barh(index[3:6], target_100met_df['byproduct_num'][3:6], height=bar_width, label='Alcohol', color='#fdbf6f')
plt.barh(index[1:3], target_100met_df['byproduct_num'][1:3], height=bar_width, label='Alkaloid', color='#ff7f00')
plt.barh(index[0:1], target_100met_df['byproduct_num'][0:1], height=bar_width, label='Amino acid', color='#cab2d6')

# Setting Axis Parameters
plt.tick_params(axis='x', direction='in', width=0.5, which='both', length=1)
plt.tick_params(axis='y', which='both', width=0.5, length=1, pad=1)
plt.ylim(-1, 44)
plt.xlim(0, 25)
plt.yticks(index, target_100met_df['Name'], fontsize=5.7)
plt.xticks(np.arange(0, 26, 5), fontsize=6)
plt.xlabel('Number of reaction', fontsize=6)

# Add legend
plt.legend(frameon=False, fontsize=7, loc='center left', bbox_to_anchor=(-1.4, 0.5), labelspacing=0.5, handletextpad=0.3, handlelength=1)

# Display and save the plot
plt.tight_layout()
plt.savefig(yeast8U_heterologous_product_reaction_figure_path, dpi=400, bbox_inches='tight')
plt.show()